# Media total
Пример отчета по расчету аудитории по всему интернету.

## Описание задачи

Общие параметры:
- Период: Октябрь 2021
- Население: Возрастная группа 12-34
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)
- Reach% (reachPer)
- Universe (uni)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Задание

### Расчет аудитории по всему интернету

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Total Internet'

# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-10-01', '2021-10-31')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = 'ageGroup IN (1,2,3)'

# Задаем фильтр по медиа, в нашем случае он не требуется 
mart_filter = None

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "useTypeName"]

# Указываем список статистик для расчета
statistics = ["reach", "reachPer", "uni"]

scales = None


# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('total', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics, scales)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_task(task_json))

# Получаем результат
df_result = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_result

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('total-dec-2021'))
df_info = mtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()